In [3]:
import rdkit.Chem as Chem

In [1]:
import pyclops
from pyclops.core.loss_handler.chemical_loss_handler import ChemicalLossHandler

chem_handler = ChemicalLossHandler.from_pdb_file(pdb_file="//Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/chignolin.pdb", 
                                                 units_factor=1.0,
                                                 )

print(chem_handler.summary)

AmideAspHead [0, 2] (x2 resonance groups)| AmideGluHead [0, 4] (x2 resonance groups)| AmideHead2Tail [0, 9] (x2 resonance groups)| AspCTerm [2, 9] (x4 resonance groups)| AspGlu [2, 4] (x4 resonance groups)| GluCTerm [4, 9] (x4 resonance groups)
